In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.neighbors import NearestNeighbors
import pickle


In [2]:
# Load data
df = pd.read_csv(r'C:\Users\user\Desktop\Laptop_Price_Prediction_Model_Nepal\NP Laptop Price.csv')

# Convert 'Graphics_card' to numerical
df['Graphics_card'] = df['Graphics_card'].map({'No': 0, 'Yes': 1})

# Remove specific brands
brands_to_remove = ['MSI', 'MI', 'Honor']
df = df[~df['Brand'].isin(brands_to_remove)]

# Combine similar models
model_combinations = {
    'IdeaPad Slim 3': ['Ideapad Slim 3', 'IP Slim 3'],
    'Vivobook': ['Vivobook Go', 'VivoBook'],
}

def combine_models(row, combinations):
    for key, values in combinations.items():
        if row in values:
            return key
    return row

df['Model'] = df['Model'].apply(combine_models, args=(model_combinations,))

models_to_combine = [
    'V15', 'LOQ 15', 'Victus', 'ZenBook', 'V14', 'Latitude 5480', 'Nitro V',
    'GemiBook X', 'Zephyrus G15', 'Nitro V15', 'Latitude E5400', 'Swift 3',
    'Ideapad', 'Legion Slim 5', 'Swift Go', 'HeroBook Plus', 'Travelmate P2',
    'Latitude E7490', 'LOQ', 'Swift Go 14', 'Latitude E7390', 'Nitro 5',
    'Notebook 250 G9', 'Expertbook', '15 fc0093dx', 'Latitude E5300', 'IdeaPad 1',
    'Predator Helios 300', 'Predator Triton 300 SE', 'Aspire 3', 'Ideapad 1',
    'Latitude E5470', 'Envy x360', 'LOQ '
]

df['Model'] = df['Model'].apply(lambda x: 'Other' if x in models_to_combine else x)

# Filter data
df = df[df['Price'] < 160000]
df['Storage SSD'] = np.log(df['Storage SSD'])

# Drop duplicates
df = df.drop_duplicates()
df.reset_index(drop=True, inplace=True)


In [3]:
# Define categorical and numerical columns
cat_cols = ['Brand', 'Model', 'Processor']
num_cols = ['RAM', 'Storage SSD', 'Size', 'Graphics_card']

# Define preprocessing pipeline
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, num_cols),
        ('cat', categorical_transformer, cat_cols)
    ])


In [4]:
# Create NearestNeighbors model
nn_model = NearestNeighbors(n_neighbors=5, metric='euclidean')

# Create pipeline including preprocessing
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('nn', nn_model)
])

# Fit the pipeline to the data
pipeline.fit(df.drop('Price', axis=1))

# Save the recommendation model to a pickle file
with open('recommendation_model.pkl', 'wb') as f:
    pickle.dump(pipeline, f)

print("Recommendation model saved to 'recommendation_model.pkl'")


Recommendation model saved to 'recommendation_model.pkl'


In [6]:
import pandas as pd
import numpy as np
import pickle
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Load the dataset
df = pd.read_csv(r'C:\Users\user\Desktop\Laptop_Price_Prediction_Model_Nepal\NP Laptop Price.csv')

# Preprocessing steps (including combining models and handling categorical variables)
df['Graphics_card'] = df['Graphics_card'].map({'No': 0, 'Yes': 1})
brands_to_remove = ['MSI', 'MI', 'Honor']
df = df[~df['Brand'].isin(brands_to_remove)]

model_combinations = {
    'IdeaPad Slim 3': ['Ideapad Slim 3', 'IP Slim 3'],
    'Vivobook': ['Vivobook Go', 'VivoBook'],
}

def combine_models(row, combinations):
    for key, values in combinations.items():
        if row in values:
            return key
    return row

df['Model'] = df['Model'].apply(combine_models, args=(model_combinations,))

models_to_combine = [
    'V15', 'LOQ 15', 'Victus', 'ZenBook', 'V14', 'Latitude 5480', 'Nitro V',
    'GemiBook X', 'Zephyrus G15', 'Nitro V15', 'Latitude E5400', 'Swift 3',
    'Ideapad', 'Legion Slim 5', 'Swift Go', 'HeroBook Plus', 'Travelmate P2',
    'Latitude E7490', 'LOQ', 'Swift Go 14', 'Latitude E7390', 'Nitro 5',
    'Notebook 250 G9', 'Expertbook', '15 fc0093dx', 'Latitude E5300', 'IdeaPad 1',
    'Predator Helios 300', 'Predator Triton 300 SE', 'Aspire 3', 'Ideapad 1',
    'Latitude E5470', 'Envy x360', 'LOQ '
]

df['Model'] = df['Model'].apply(lambda x: 'Other' if x in models_to_combine else x)
df = df[df['Price'] < 160000]
df['Storage SSD'] = np.log(df['Storage SSD'])
df = df.drop_duplicates()
df.reset_index(drop=True, inplace=True)

# Define features and target
X = df.drop('Price', axis=1)
y = df['Price']

# Categorical and numerical columns
cat_cols = ['Brand', 'Model', 'Processor']
num_cols = ['RAM', 'Storage SSD', 'Size', 'Graphics_card']

# Define preprocessing pipeline
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, num_cols),
        ('cat', categorical_transformer, cat_cols)
    ])

# Save the preprocessor
with open('preprocessor.pkl', 'wb') as f:
    pickle.dump(preprocessor, f)
    
print("Preprocessor saved to 'preprocessor.pkl'")

# Save the DataFrame for recommendation purposes
df.to_pickle('dataframe.pkl')
print("DataFrame saved to 'dataframe.pkl'")


Preprocessor saved to 'preprocessor.pkl'
DataFrame saved to 'dataframe.pkl'
